In [45]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [46]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [47]:
template = {
    'maf': {
            'class': 'File',
            'path': 'a/file/path'
    },
    'bam': {
            'class': 'File',
            'path': 'a/file/path'
    },
    'ref_dir': {
        'class': 'Directory',
        'path': 'a/dir/path'
    },
    'bed': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'input_type': 'a_string' # rna or dna, default is dna
}

In [48]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.neoscan.yaml'), 'w'))

In [49]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'neoscan',
    'label': 'neoscan',
    'baseCommand': ['/usr/bin/python', '/pecgs-neoscan/src/neoscan.py'],
    'inputs': [
        {
            'id': 'maf',
            'type': 'File',
            'inputBinding': {
                'position': '1',
            }
        },
        {
            'id': 'bam',
            'type': 'File',
            'secondaryFiles': [
               '$(self.basename).bai',
            ],
            'inputBinding': {
                'position': '2',
            }
        },
        {
            'id': 'ref_dir',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--ref-dir',
                'position': '0'
            }
        },
        {
            'id': 'bed',
            'type': 'File',
            'inputBinding': {
                'prefix': '--bed',
                'position': '0'
            }
        },
        {
            'id': 'input_type',
            'type': 'string?',
            'default': 'dna',
            'inputBinding': {
                'prefix': '--input-type',
                'position': '0'
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
#             'default': '/miniconda/envs/neoscan/bin:$PATH'
            'default': '/usr/local/bin/OptiType:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/home/biodocker/bin:/miniconda/envs/neoscan/bin:/miniconda/bin:$PATH' # just hardcoding here is the easiest thing for now
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--out-dir',
            'valueFrom': 'output'
        },
        {
            'position': 0,
            'prefix': '--log-dir',
            'valueFrom': 'logs'
        },
        {
            'position': 0,
            'prefix': '--neoscan-dir',
            'valueFrom': '/pecgs-neoscan/src/neoscan'
        },
        {
            'position': 0,
            'prefix': '--optitype-script',
            'valueFrom': '/usr/local/bin/OptiType/OptiTypePipeline.py'
        },
        {
            'position': 0,
            'prefix': '--f-opti-config',
            'valueFrom': '/pecgs-neoscan/src/neoscan/config.ini'
        },
        {
            'position': 0,
            'prefix': '--netmhc',
            'valueFrom': '/pecgs-neoscan/src/netmhc/netMHC-4.0/netMHC'
        },
                {
            'position': 0,
            'prefix': '--f-allele',
            'valueFrom': '/pecgs-neoscan/src/netmhc/netMHC-4.0/data/allelelist'
        },
        
    ],
    'outputs': [
        {
            'id': 'snv_summary',
            'type': 'File',
            'outputBinding': {'glob': 'output/sample.neo.snv.summary'},
        },
        {
            'id': 'indel_summary',
            'type': 'File',
            'outputBinding': {'glob': 'output/sample.neo.indel.summary'},
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs-neoscan:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 100000,
            'coresMin': 4
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [50]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'neoscan.cwl'), 'w'))

In [ ]:
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"

In [ ]:
cd /pecgs-neoscan/src/neoscan
export PATH="/miniconda/envs/neoscan/bin:$PATH"

In [ ]:
perl neoscan.pl --rdir /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/execution/output --bed /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/inputs/1290633107/proteome.bed --refdir /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/inputs/881998829/refseq_hg38_june29 --optitype /miniconda/envs/neoscan/bin/OptiPathPipeline.py --fallele /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/inputs/737207470/allelelist --netmhc /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/inputs/-214520106/netMHC --fopticonfig /pecgs-neoscan/src/neoscan/config.ini --bam 1 --rna 0 --log /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/cromwell-executions/neoscan.cwl/6994fb4a-0450-4dbb-818a-bcfdc8d73721/call-neoscan.cwl/execution/logs --step 4

